<a href="https://colab.research.google.com/github/vbreddy007/WebScraper/blob/main/bg_trransformers.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!git clone https://huggingface.co/briaai/RMBG-1.4
%cd RMBG-1.4/
!pip install -r requirements.txt

fatal: destination path 'RMBG-1.4' already exists and is not an empty directory.
/content/RMBG-1.4


Exception in thread Thread-5 (attachment_entry):
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/debugpy/server/api.py", line 237, in listen
    sock, _ = endpoints_listener.accept()
  File "/usr/lib/python3.10/socket.py", line 293, in accept
    fd, addr = self._accept()
TimeoutError: timed out

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/lib/python3.10/threading.py", line 1016, in _bootstrap_inner
    self.run()
  File "/usr/lib/python3.10/threading.py", line 953, in run
    self._target(*self._args, **self._kwargs)
  File "/usr/local/lib/python3.10/dist-packages/google/colab/_debugpy.py", line 52, in attachment_entry
    debugpy.listen(_dap_port)
  File "/usr/local/lib/python3.10/dist-packages/debugpy/public_api.py", line 31, in wrapper
    return wrapped(*args, **kwargs)
  File "/usr/local/lib/python3.10/dist-packages/debugpy/server/api.py", line 143, in debug
    log.reraise

In [2]:
from skimage import io
import torch, os
from PIL import Image
import sys
sys.path.append('/content/RMBG-1.4/RMBG-1.4')
from briarmbg import BriaRMBG
from utilities import preprocess_image, postprocess_image

In [3]:
im_path = "/content/drive/MyDrive/newDP.jpg"
im_path2 = "/content/drive/MyDrive/photo.PNG"


In [4]:
net = BriaRMBG()
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
net = BriaRMBG.from_pretrained("briaai/RMBG-1.4")
net.to(device)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


pytorch_model.bin:   0%|          | 0.00/177M [00:00<?, ?B/s]

BriaRMBG(
  (conv_in): Conv2d(3, 64, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
  (pool_in): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=True)
  (stage1): RSU7(
    (rebnconvin): REBNCONV(
      (conv_s1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (bn_s1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu_s1): ReLU(inplace=True)
    )
    (rebnconv1): REBNCONV(
      (conv_s1): Conv2d(64, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (bn_s1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu_s1): ReLU(inplace=True)
    )
    (pool1): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=True)
    (rebnconv2): REBNCONV(
      (conv_s1): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (bn_s1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu_s1): ReL

In [11]:
im_path = "/content/drive/MyDrive/newDP.jpg"
im_path2 = "/content/drive/MyDrive/g.jpg"
im_path3 = "/content/drive/MyDrive/IMG-2772.jpg"

In [12]:
model_input_size = [1024,1024]
orig_im = io.imread(im_path3)
orig_im_size = orig_im.shape[0:2]
image = preprocess_image(orig_im, model_input_size).to(device)

In [27]:
# inference
result=net(image)

# post process
result_image = postprocess_image(result[0][0], orig_im_size)

In [13]:
# prompt: caluclate the inference and memory usage during inference

import time
import torch
import gc

# Calculate inference time
start_time = time.time()
result = net(image)
inference_time = time.time() - start_time

# Calculate memory usage
max_memory_usage = 0
for tensor in net.parameters():
    max_memory_usage = max(max_memory_usage, tensor.numel() * tensor.element_size())
max_memory_usage /= 1024 ** 2  # Convert bytes to megabytes

# Print results
print(f"Inference time: {inference_time:.2f} seconds")
print(f"Maximum memory usage: {max_memory_usage:.2f} MB")

# post process
result_image = postprocess_image(result[0][0], orig_im_size)



Inference time: 0.04 seconds
Maximum memory usage: 18.00 MB


In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [15]:
 #save result
pil_im = Image.fromarray(result_image)
no_bg_image = Image.new("RGBA", pil_im.size, (0,0,0,0))
orig_image = Image.open(im_path3)
no_bg_image.paste(orig_image, mask=pil_im)
no_bg_image.save("/content/drive/MyDrive/example_image_no_bg3.png")


In [43]:
# prompt: caluclate the memory usage after clearing memory and before clearing memory
# Calculate memory usage before clearing memory
max_memory_usage_before_clear = 0
for tensor in net.parameters():
    max_memory_usage_before_clear = max(max_memory_usage_before_clear, tensor.numel() * tensor.element_size())
max_memory_usage_before_clear /= 1024 ** 2  # Convert bytes to megabytes

del image
del result
del result_image


# Calculate memory usage after clearing memory
gc.collect()
max_memory_usage_after_clear = 0
for tensor in net.parameters():
    max_memory_usage_after_clear = max(max_memory_usage_after_clear, tensor.numel() * tensor.element_size())
max_memory_usage_after_clear /= 1024 ** 2  # Convert bytes to megabytes



# Print results
print(f"Maximum memory usage before clearing: {max_memory_usage_before_clear:.2f} MB")
print(f"Maximum memory usage after clearing: {max_memory_usage_after_clear:.2f} MB")


Maximum memory usage before clearing: 18.00 MB
Maximum memory usage after clearing: 18.00 MB
